1. Quadratic equation solver

This part of project demonstrates conditional workflows in LangGraph through a quadratic equation solver, where execution paths dynamically branch based on discriminant values. Note goal here is to first see how to create conditional edges and later in 2nd part will make more complex graph with LLM

In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal

In [2]:
# let's create a state type for our graph
class QuadState(TypedDict):
    a: int
    b: int
    c: int

    equation: str
    discriminant: float
    result: str

In [3]:
# nodes
def show_equation(state: QuadState):

    equation = f'{state["a"]}x2{state["b"]}x{state["c"]}'

    return {'equation':equation}

def calculate_discriminant(state: QuadState):

    discriminant = state["b"]**2 - (4*state["a"]*state["c"])

    return {'discriminant': discriminant}

def real_roots(state: QuadState):

    root1 = (-state["b"] + state["discriminant"]**0.5)/(2*state["a"])
    root2 = (-state["b"] - state["discriminant"]**0.5)/(2*state["a"])

    result = f'The roots are {root1} and {root2}'

    return {'result': result}

def repeated_roots(state: QuadState):

    root = (-state["b"])/(2*state["a"])

    result = f'Only repeating root is {root}'

    return {'result': result}

def no_real_roots(state: QuadState):

    result = f'No real roots'

    return {'result': result}

# for conditional branching, we need to return a literal string
# that matches the name of the next node
# kind of routing node
def check_condition(state: QuadState) -> Literal["real_roots", "repeated_roots", "no_real_roots"]:

    if state['discriminant'] > 0:
        return "real_roots"
    elif state['discriminant'] == 0:
        return "repeated_roots"
    else:
        return "no_real_roots"

In [4]:
graph = StateGraph(QuadState)

graph.add_node('show_equation', show_equation)
graph.add_node('calculate_discriminant', calculate_discriminant)
graph.add_node('real_roots', real_roots)
graph.add_node('repeated_roots', repeated_roots)
graph.add_node('no_real_roots', no_real_roots)


graph.add_edge(START, 'show_equation')
graph.add_edge('show_equation', 'calculate_discriminant')

graph.add_conditional_edges('calculate_discriminant', check_condition)
graph.add_edge('real_roots', END)
graph.add_edge('repeated_roots', END)
graph.add_edge('no_real_roots', END)

workflow = graph.compile()

# visualize
try:
    from IPython.display import Image
    Image(workflow.get_graph().draw_mermaid_png())
except:
    pass

In [5]:
# we can visualize the graph using mermaid syntax
# https://mermaid.live/
# sometime due to network issues, image may not render, in that case use mermaid_code below to visualize

mermaid_code = workflow.get_graph().draw_mermaid()
print(mermaid_code)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	show_equation(show_equation)
	calculate_discriminant(calculate_discriminant)
	real_roots(real_roots)
	repeated_roots(repeated_roots)
	no_real_roots(no_real_roots)
	__end__([<p>__end__</p>]):::last
	__start__ --> show_equation;
	calculate_discriminant -.-> no_real_roots;
	calculate_discriminant -.-> real_roots;
	calculate_discriminant -.-> repeated_roots;
	show_equation --> calculate_discriminant;
	no_real_roots --> __end__;
	real_roots --> __end__;
	repeated_roots --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [6]:
initial_state = {
    'a': 2, 
    'b': 4,
    'c': 2
}

final_state = workflow.invoke(initial_state)

final_state

{'a': 2,
 'b': 4,
 'c': 2,
 'equation': '2x24x2',
 'discriminant': 0,
 'result': 'Only repeating root is -1.0'}

2. Review Reply Workflow

An LLM-powered conditional workflow in LangGraph that analyzes customer reviews and dynamically generates context-aware replies based on sentiment and content.

In [7]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field
load_dotenv()

True

In [8]:
# let's define schemas for sentiment analysis and diagnosis

class SentimentSchema(BaseModel):

    # Literal is used when you want a variable (or function parameter) to accept only specific, 
    # fixed values instead of any value of a type.
    # Here, we restrict sentiment to be either "positive" or "negative"
    sentiment: Literal["positive", "negative"] = Field(description='Sentiment of the review')

class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')

In [9]:
# for structured output, we need to create a structured model
model = ChatOpenAI(model='gpt-4o-mini')
structured_model = model.with_structured_output(SentimentSchema)
structured_model2 = model.with_structured_output(DiagnosisSchema)

In [10]:
# define State
class ReviewState(TypedDict):
    review: str
    sentiment: Literal["positive", "negative"]
    diagnosis: dict
    response: str

In [11]:
# define nodes

def find_sentiment(state: ReviewState):

    prompt = f'For the following review find out the sentiment \n {state["review"]}'
    sentiment = structured_model.invoke(prompt).sentiment

    return {'sentiment': sentiment}

# conditional branching node
def check_sentiment(state: ReviewState) -> Literal["positive_response", "run_diagnosis"]:

    if state['sentiment'] == 'positive':
        return 'positive_response'
    else:
        return 'run_diagnosis'

def positive_response(state: ReviewState):

    prompt = f"""Write a warm thank-you message in response to this review:
    \n\n\"{state['review']}\"\n
Also, kindly ask the user to leave feedback on our website."""
    
    response = model.invoke(prompt).content

    return {'response': response}

def run_diagnosis(state: ReviewState):

    prompt = f"""Diagnose this negative review:\n\n{state['review']}\n"
    "Return issue_type, tone, and urgency.
"""
    response = structured_model2.invoke(prompt)

    # response is in json, so we convert to dict
    return {'diagnosis': response.model_dump()}

def negative_response(state: ReviewState):

    diagnosis = state['diagnosis']

    prompt = f"""You are a support assistant.
The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'.
Write an empathetic, helpful resolution message. Just write response message, no subjectes or greetings.
"""
    response = model.invoke(prompt).content

    return {'response': response}
    

In [12]:
# make graph, add nodes and edges
graph = StateGraph(ReviewState)

graph.add_node('find_sentiment', find_sentiment)
graph.add_node('positive_response', positive_response)
graph.add_node('run_diagnosis', run_diagnosis)
graph.add_node('negative_response', negative_response)

graph.add_edge(START, 'find_sentiment')

graph.add_conditional_edges('find_sentiment', check_sentiment)

graph.add_edge('positive_response', END)

graph.add_edge('run_diagnosis', 'negative_response')
graph.add_edge('negative_response', END)

workflow = graph.compile()

In [13]:
# visualize
try:
    from IPython.display import Image
    Image(workflow.get_graph().draw_mermaid_png())
except:
    pass

In [14]:
# we can visualize the graph using mermaid syntax
# https://mermaid.live/
# sometime due to network issues, image may not render, in that case use mermaid_code below to visualize

mermaid_code = workflow.get_graph().draw_mermaid()
print(mermaid_code)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	find_sentiment(find_sentiment)
	positive_response(positive_response)
	run_diagnosis(run_diagnosis)
	negative_response(negative_response)
	__end__([<p>__end__</p>]):::last
	__start__ --> find_sentiment;
	find_sentiment -.-> positive_response;
	find_sentiment -.-> run_diagnosis;
	run_diagnosis --> negative_response;
	negative_response --> __end__;
	positive_response --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [15]:
intial_state={
    'review': "I’ve been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality."
}

final_state = workflow.invoke(intial_state)

final_state

{'review': 'I’ve been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality.',
 'sentiment': 'negative',
 'diagnosis': {'issue_type': 'Bug', 'tone': 'frustrated', 'urgency': 'high'},
 'response': "I understand how frustrating it can be to deal with a bug, especially when it impacts your work. I appreciate your patience as we work to resolve this. To help us tackle the issue effectively, could you please provide me with a few details? Specifically, let me know what the bug is, any error messages you're seeing, and the steps you've taken so far. This information will help me assist you more efficiently. Thank you for your understanding, and I’m here to help get this sorted out for you as quickly as possible."}

In [16]:
# let's see response in cleaner format
from IPython.display import Markdown, display
display(Markdown(final_state['response']))

I understand how frustrating it can be to deal with a bug, especially when it impacts your work. I appreciate your patience as we work to resolve this. To help us tackle the issue effectively, could you please provide me with a few details? Specifically, let me know what the bug is, any error messages you're seeing, and the steps you've taken so far. This information will help me assist you more efficiently. Thank you for your understanding, and I’m here to help get this sorted out for you as quickly as possible.